In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [3]:
url = 'https://content.codecademy.com/courses/beautifulsoup/cacao/index.html'

page = requests.get(url)
soup = BeautifulSoup(page.text, 'html.parser')

In [4]:
#create column names for the DataFrame from the table header
column_names = []
all_tr_tag = soup.find_all('tr')

headers = all_tr_tag[2].find_all('td')
for td in headers:
  column_names.append(td.get_text())

final_column_names = []
for name in column_names:
  name = name.strip()
  name = name.replace('\n', "")
  #the code below will remove all spaces and leave the string with one word one space
  name = ' '.join(name.split())
  final_column_names.append(name.strip())

# print(final_column_names) 

final_df = pd.DataFrame(columns=final_column_names)

In [34]:
#put all ratings into a list
rating_list = []
ratings = soup.find_all('td', attrs = {'class': 'Rating'})
for rating in ratings[1:]:
  rating_list.append(rating.get_text())
#convert strings into floats
final_rating_list = [float(i) for i in rating_list]
#print(final_rating_list)

In [6]:
#put all companies in a list
company_list = []
companies = soup.find_all('td', attrs = {'class': 'Company'})
for company in companies[1:]:
  company_list.append(company.get_text())
#print(company_list)

In [7]:
#put all bean origin in a list
bean_origin = []
bean_origins = soup.find_all('td', attrs = {'class': 'Origin'})
for origin in bean_origins[1:]:
  bean_origin.append(origin.get_text())
#print(bean_origin)

In [8]:
ref_list = []
refs = soup.find_all('td', attrs = {'class': 'REF'})
for year in refs[1:]:
  ref_list.append(int(year.get_text()))

In [9]:
#all review date in a list
review_date_list = []
review_dates = soup.find_all('td', attrs = {'class': 'ReviewDate'})
for dates in review_dates[1:]:
  review_date_list.append(int(dates.get_text()))
#print(review_date_list)

In [10]:
#all cocoa percent in a list
cocoa_percent_list = []
cocoa_percents = soup.find_all('td', attrs = {'class': 'CocoaPercent'})
for percent in cocoa_percents[1:]:
  percent = percent.get_text()
  percent = percent.replace('%', '')
  cocoa_percent_list.append(float(percent))
#print(cocoa_percent_list)

In [11]:
#all locations in a list
location_list = []
locations = soup.find_all('td', attrs = {'class': 'CompanyLocation'})
for location in locations[1:]:
  location_list.append(location.get_text())
#print(location_list)

In [12]:
#all bean types in a list
bean_type_list = []
bean_types = soup.find_all('td', attrs = {'class': 'BeanType'})
for bean_type in bean_types[1:]:
  bean_type = bean_type.get_text()
  bean_type = bean_type.strip()
  bean_type_list.append(bean_type)
#print(bean_type_list)

In [13]:
#all broad bean origin in a list
broad_bean_origin_list = []
broad_bean_origins = soup.find_all('td', attrs = {'class': 'BroadBeanOrigin'})
for origin in broad_bean_origins[1:]:
  origin = origin.get_text()
  origin = origin.strip()
  broad_bean_origin_list.append(origin)
#print(broad_bean_origin_list)

In [22]:
cacao_dict = {
  'company': company_list, 
  'specific_bean_origin_or_bar_name': bean_origin, 
  'ref': ref_list, 
  'review_date': review_date_list, 
  'cocoa_percent': cocoa_percent_list, 
  'company_location': location_list, 
  'rating': rating_list,
  'bean_type': bean_type_list,
  'broad_bean_origin': broad_bean_origin_list}
#print(cacao_dict)

In [28]:
cacao_df= pd.DataFrame.from_dict(cacao_dict)
cacao_df.head()

,company,specific_bean_origin_or_bar_name,ref,review_date,cocoa_percent,company_location,rating,bean_type,broad_bean_origin
0,A. Morin,Agua Grande,1876,2016,63.0,France,3.75,,Sao Tome
1,A. Morin,Kpime,1676,2015,70.0,France,2.75,,Togo
2,A. Morin,Atsane,1676,2015,70.0,France,3,,Togo
3,A. Morin,Akata,1680,2015,70.0,France,3.5,,Togo
4,A. Morin,Quilla,1704,2015,70.0,France,3.5,,Peru


In [32]:
#top 10 highest rated chocolatiers
##by using dtype(), I found out that rating column's type is object, so in order to calculate the mean, 
##I need to convert the column with .to_numeric()
cacao_df1 = pd.to_numeric(cacao_df['rating']).groupby(cacao_df['company'])
cacao_df1.mean().nlargest(10)

company
Tobago Estate (Pralus)                  4.000000
Heirloom Cacao Preservation (Zokoko)    3.875000
Ocelot                                  3.875000
Amedei                                  3.846154
Matale                                  3.812500
Patric                                  3.791667
Idilio (Felchlin)                       3.775000
Acalli                                  3.750000
Chocola'te                              3.750000
Christopher Morel (Felchlin)            3.750000
Name: rating, dtype: float64

In [35]:
cacao_df.groupby('company').rating.mean()

DataError: No numeric types to aggregate

In [33]:
#is more cocoa better?
cacao_df2 = pd.to_numeric(cacao_df['rating']).groupby(cacao_df['cocoa_percent'])
cacao_df2.mean().nlargest(10)

cocoa_percent
50.0    3.750000
63.0    3.604167
69.0    3.500000
66.0    3.380435
67.0    3.351852
78.0    3.338235
68.0    3.287234
70.0    3.276042
56.0    3.250000
86.0    3.250000
Name: rating, dtype: float64